In [1]:
import pandas as pd

In [2]:
# datapoints
# steps:
# - combine all files from source dir
# - manage person renaming
# - groupby person, interpolate datapoints for missing years

In [3]:
import sys
sys.path.insert(0, '../scripts/')

In [4]:
import etllib

In [5]:
%load_ext autoreload
%autoreload 1
%aimport etllib

In [6]:
%aimport

Modules to reload:
etllib

Modules to skip:



In [7]:
# step 1

res = []

for year in range(1997, 2022):
    df = etllib.get_data_file(year)
    df['year'] = year
    res.append(df)

res = pd.concat(res, ignore_index=True)

In [8]:
res

,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,country,...,squareImage,worth,position,rank,year,title,government,salary,managementAssets,pay
0,A. Jayson Adair,Adair,a-jayson-adair,no-pic,-32.566,52.0,damaged cars,Automotive,M,United States,...,https://specials-images.forbesimg.com/imageser...,NaN,NaN,NaN,1997,NaN,NaN,NaN,NaN,NaN
1,Abdulla Al Futtaim & family,Al Futtaim,abdulla-al-futtaim,abdulla-al-futtaim,-4.939,NaN,"auto dealers, investments",Automotive,M,United Arab Emirates,...,//specials-images.forbesimg.com/imageserve/5ab...,NaN,NaN,NaN,1997,NaN,NaN,NaN,NaN,NaN
2,Abdulla bin Ahmad Al Ghurair & family,Al Ghurair,abdulla-bin-ahmad-al-ghurair,abdulla-bin-ahmad-al-ghurair,0.000,NaN,diversified,Diversified,M,United Arab Emirates,...,//specials-images.forbesimg.com/imageserve/5a9...,NaN,NaN,NaN,1997,NaN,NaN,NaN,NaN,NaN
3,Abdulsamad Rabiu,Rabiu,abdulsamad-rabiu,abdulsamad-rabiu,-212.219,61.0,"cement, sugar",Diversified,M,Nigeria,...,https://specials-images.forbesimg.com/imageser...,NaN,NaN,NaN,1997,NaN,NaN,NaN,NaN,NaN
4,Abhay Firodia,Firodia,abhay-firodia,abhay-firodia,0.000,77.0,automobiles,Automotive,M,India,...,https://specials-images.forbesimg.com/imageser...,NaN,NaN,NaN,1997,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71266,Daniel Yong Zhang,Zhang,daniel-yong-zhang-1,no-pic,0.000,50.0,e-commerce,Technology,M,China,...,NaN,1000.0,2751.0,2674.0,2021,NaN,NaN,NaN,NaN,NaN
71267,Zhang Yuqiang,Zhang,zhang-yuqiang,no-pic,NaN,66.0,Fiberglass,Manufacturing,M,China,...,NaN,1000.0,2752.0,2674.0,2021,NaN,NaN,NaN,NaN,NaN
71268,Zhao Meiguang,Zhao,zhao-meiguang,no-pic,NaN,59.0,gold mining,Metals & Mining,M,China,...,NaN,1000.0,2753.0,2674.0,2021,NaN,NaN,NaN,NaN,NaN
71269,Zhong Naixiong,Zhong,zhong-naixiong,no-pic,NaN,59.0,conglomerate,Diversified,M,China,...,NaN,1000.0,2754.0,2674.0,2021,NaN,NaN,NaN,NaN,NaN


In [9]:
res[res.uri.str.contains("daniel-yong-zhang")]['uri'].unique()

array(['daniel-yong-zhang-1'], dtype=object)

In [10]:
from ddf_utils.str import to_concept_id

In [11]:
res.columns

Index(['name', 'lastName', 'uri', 'imageUri', 'worthChange', 'age', 'source',
       'industry', 'gender', 'country', 'timestamp', 'headquarters', 'state',
       'realTimeWorth', 'realTimeRank', 'realTimePosition', 'squareImage',
       'worth', 'position', 'rank', 'year', 'title', 'government', 'salary',
       'managementAssets', 'pay'],
      dtype='object')

In [12]:
# assuming the uri value is unique for each person
# we don't use names because they are not consistent.

res['person'] = res['uri'].map(to_concept_id)

In [13]:
dps = res[['person', 'year', 'worth']].copy()

In [14]:
gr = dps.groupby('person')


In [15]:
d = gr.get_group('zong_qinghou')

In [16]:
ymax = d.year.max()
ymin = d.year.min()
d_ = d.set_index('year').reindex(range(ymin, ymax+1))

In [17]:
d_.worth.interpolate()

year
1997     8758.893000
1998     8758.893000
1999     8758.893000
2000     8758.893000
2001     8758.893000
2002     8758.893000
2003     8758.893000
2004     8758.893000
2005     6205.928667
2006     3652.964333
2007     1100.000000
2008     1000.000000
2009     1900.000000
2010     7000.000000
2011     5900.000000
2012     6500.000000
2013    11600.000000
2014    11600.000000
2015    10300.000000
2016     6200.000000
2017     7200.000000
2018     9500.000000
2019     8400.000000
2020     8300.000000
2021     8800.000000
Name: worth, dtype: float64

In [18]:
def interpolate(frame):
    ymax = frame.year.max()
    ymin = frame.year.min()
    df = frame.set_index('year').reindex(range(ymin, ymax+1))
    df.worth.interpolate(inplace=True)
    return df.reset_index()

In [19]:
dps2 = gr.apply(interpolate)

/tmp/ipykernel_108255/2986774926.py:4: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  df = frame.set_index('year').reindex(range(ymin, ymax+1))


ValueError: cannot reindex on an axis with duplicate labels

In [21]:
import numpy as np

In [22]:
for g, d in gr:
    if np.any(d.year.duplicated()):
        print(g)
        

oleg_deripaska


In [23]:
gr.get_group('oleg_deripaska')

,person,year,worth
1595,oleg_deripaska,1997,4097.979
3738,oleg_deripaska,1998,4097.979
6861,oleg_deripaska,1999,NaN
9457,oleg_deripaska,2000,NaN
11905,oleg_deripaska,2001,NaN
15653,oleg_deripaska,2002,1100.000
18205,oleg_deripaska,2003,1500.000
20840,oleg_deripaska,2004,3300.000
23576,oleg_deripaska,2005,5500.000
26272,oleg_deripaska,2006,7800.000


In [24]:
res[(res.person == 'oleg_deripaska') & (res.year == 2016)][['name', 'age', 'industry', 'worth']]

,name,age,industry,worth
55597,Oleg Deripaska,54.0,Metals & Mining,2100.0
55598,Oleg Deripaska,54.0,Metals & Mining,2100.0


In [25]:
# it's the same, let's drop it..

def interpolate(frame):
    ymax = frame.year.max()
    ymin = frame.year.min()
    if np.any(frame.year.duplicated()):
        person = frame.person.iloc[0]
        print(f'{person} has duplicated datapoints. I will be drop the duplicated ones. Please double check.')
        frame = frame.drop_duplicates(subset=['year'])
    df = frame.set_index('year').reindex(range(ymin, ymax+1))
    df.worth.interpolate(inplace=True)
    return df.worth

In [26]:
dps2 = gr.apply(interpolate)

oleg_deripaska has duplicated datapoints. I will be drop the duplicated ones. Please double check.


In [27]:
dps2

person              year
a_jayson_adair      1997       NaN
                    1998       NaN
                    1999       NaN
                    2000       NaN
                    2001       NaN
                             ...  
zygmunt_solorz_zak  2017    2500.0
                    2018    2800.0
                    2019    2700.0
                    2020    2400.0
                    2021    3200.0
Name: worth, Length: 71843, dtype: float64

In [28]:
dps2 = dps2.dropna()

In [29]:
dps2.to_csv('../../ddf--datapoints--worth--by--person--year.csv')

In [30]:
# TODO: maybe also add age
# ages = res[['person', 'age']].drop_duplicates().dropna()

In [37]:
# assume average annual return on assets is 3%, calculate the income for all data points
r = 0.03

income = dps2 * r * 1e6

In [38]:
income.name = 'annual_income'

income.to_csv('../../ddf--datapoints--annual_income--by--person--year.csv')

In [39]:
# lookup Bezoe

income.loc['jeff_bezos', 2021]

5310000000.0

In [54]:
b = income.loc['jeff_bezos', 2021]

In [65]:
def get_level(n):
    i = 1
    acc = n / 2
    while acc > 1:
        acc = acc / 4
        i = i + 1
    return i

In [66]:
get_level(b)

17

In [60]:
def test_cal(n):
    return 2 * (4**(n-1))

In [53]:
test_cal(17)

34359738368

In [59]:
print(test_cal(15))
print(b)

2147483648
5310000000.0


In [61]:
test_cal(1)

2

In [70]:
test_val = 15
n = get_level(test_val)

print(test_val)
print(n, test_cal(n-1), test_cal(n))

assert test_cal(n-1) <= test_val
assert test_cal(n) >= test_val


15
3 8 32


In [71]:
income_level = income.map(get_level)

In [72]:
income_level.name = "income_level"

In [73]:
income_level.to_csv('../../ddf--datapoints--income_level--by--person--year.csv')

In [71]:
# person entity
res.columns

Index(['name', 'lastName', 'uri', 'imageUri', 'worthChange', 'age', 'source',
       'industry', 'gender', 'country', 'timestamp', 'headquarters', 'state',
       'realTimeWorth', 'realTimeRank', 'realTimePosition', 'squareImage',
       'worth', 'position', 'rank', 'year', 'title', 'government', 'salary',
       'managementAssets', 'pay', 'person'],
      dtype='object')

In [73]:
person = res[['person', 'name', 'age',  'gender', 'country', 'source', 'industry']].copy().set_index('person')

In [77]:
def cleanup(s):
    if isinstance(s, str):
        s_new = s.strip()
        if "&#38;" in s_new:
            s_new = s_new.replace("&#38;", '')
        return s_new
    else:
        return s

for c in person.columns:
    person[c] = person[c].map(cleanup)

In [78]:
# merge info

ent1 = person[['name', 'country', 'source', 'industry']]
ent1 = ent1.groupby('person').apply(etllib.combine_values)

In [79]:
ent1

,name,country,source,industry
person,,,,
a_jayson_adair,A. Jayson Adair,United States,damaged cars,Automotive
a_jerrold_perenchio,A. Jerrold Perenchio,United States,"Univision; television; television, Univision",Media; Media & Entertainment
abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,banking,Finance
abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,"retail, real estate",Fashion & Retail
abdul_rasyid,Abdul Rasyid,Indonesia,"timber, palm oil",Manufacturing
...,...,...,...,...
ziyavudin_magomedov,Ziyavudin Magomedov,Russia,"port, gas",Energy
zong_qinghou,Zong Qinghou; QInghou Zong,China,beverages; Beverages,Food & Beverage; Beverages; Food and Beverage
zugen_ni,Zugen Ni,China,appliances,Manufacturing


In [80]:
ent2 = person[['age', 'gender']]

In [81]:
ent2 = ent2.groupby('person').apply(etllib.get_last_value)

In [82]:
ent2

,age,gender
person,,
a_jayson_adair,52.0,M
a_jerrold_perenchio,91.0,M
abdul_aziz_al_ghurair,67.0,M
abdul_majeed_alhokair,52.0,M
abdul_rasyid,64.0,M
...,...,...
ziyavudin_magomedov,53.0,M
zong_qinghou,76.0,M
zugen_ni,65.0,M


In [83]:
ent = pd.concat([ent1, ent2],  axis=1)

In [84]:
ent

,name,country,source,industry,age,gender
person,,,,,,
a_jayson_adair,A. Jayson Adair,United States,damaged cars,Automotive,52.0,M
a_jerrold_perenchio,A. Jerrold Perenchio,United States,"Univision; television; television, Univision",Media; Media & Entertainment,91.0,M
abdul_aziz_al_ghurair,Abdul Aziz Al Ghurair,United Arab Emirates,banking,Finance,67.0,M
abdul_majeed_alhokair,Abdul Majeed Alhokair,Saudi Arabia,"retail, real estate",Fashion & Retail,52.0,M
abdul_rasyid,Abdul Rasyid,Indonesia,"timber, palm oil",Manufacturing,64.0,M
...,...,...,...,...,...,...
ziyavudin_magomedov,Ziyavudin Magomedov,Russia,"port, gas",Energy,53.0,M
zong_qinghou,Zong Qinghou; QInghou Zong,China,beverages; Beverages,Food & Beverage; Beverages; Food and Beverage,76.0,M
zugen_ni,Zugen Ni,China,appliances,Manufacturing,65.0,M


In [85]:
ent[['name', 'age',  'gender', 'country', 'source', 'industry']].to_csv('../../ddf--entities--person.csv')